In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('drive/My Drive/Colab Notebooks/UMich/SI670/Final_Proj')

Mounted at /content/drive


In [2]:
!ls

 calorie_regression_baseline.ipynb
 calorie_regression_basic_densenet121.ipynb
 calorie_regression_multiin_101_multiout.ipynb
 calorie_regression_multiin_emb_multiout_densenet121.ipynb
 calorie_regression_multiin_multiout_densenet121.ipynb
 calorie_regression_multiout_densenet121.ipynb
 clean_recipe_data_nutrition.csv
'EfficientNetV2B0 calorie_regression_multiin_101_multiout.ipynb'
 file.txt
 food-101
 food101_id2label.json
 food101_label2id.json
 food101_nutritionix.csv
 food-101.tar.gz
 get_recipe_images.ipynb
 huggingface_food101_swin.ipynb
 initial_multiin_101_multiout_densenet.h5
 initial_multiin_101_multiout_efficientnetb0.h5
 initial_multiin_101_multiout_efficientnetv2b0.h5
 initial_multiin_101_multiout_mobilenet.h5
 initial_multiin_101_multiout_NASNetMobile.h5
 initial_multiin_multiout_densenet_2.h5
 initial_multiin_multiout_densenet.h5
'mobileNetV2 calorie_regression_multiin_101_multiout.ipynb'
 My_suffering.ipynb
'NASNetMobile calorie_regression_multiin_101_multiout.ipynb'
 n

In [3]:
from google.colab import runtime
import pandas as pd
from tqdm import tqdm
import requests
import cv2
import numpy as np
import tensorflow as tf
from keras.applications.densenet import DenseNet121, preprocess_input
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten, GlobalAveragePooling2D, Dropout, concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import EarlyStopping
from PIL import Image
import zipfile
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [4]:
SEED = 670

# Load Data and Preprocess

Basic preprocessing, remove unneeded columns, replace NAs with UNK

In [5]:
data = pd.read_csv('clean_recipe_data_nutrition.csv')
data = data[
    ['id', 'servings', 'calories_per_serving', 'protein_per_serving', 
     'fat_per_serving', 'carb_per_serving', 'meal_type', 'region']
]
data['meal_type'] = data['meal_type'].fillna(value='UNK_meal_type')
data.head()

,id,servings,calories_per_serving,protein_per_serving,fat_per_serving,carb_per_serving,meal_type,region
0,1,2,388,17,22,29,main,latin_american
1,3,10,795,16,39,96,dessert,latin_american
2,4,12,384,17,25,25,main,latin_american
3,8,8,381,34,22,9,UNK_meal_type,latin_american
4,9,12,427,18,12,63,main,latin_american


In [6]:
# there are multiple meal types associated with some recipes
# assign them to only one
fix_meal_type_dict = {
    'UNK_meal_type':'UNK_meal_type', 
    'appetizer': 'appetizer', 
    'appetizer,bread': 'bread',
    'appetizer,bread,side': 'bread', 
    'appetizer,dessert': 'dessert', 
    'appetizer,main': 'appetizer',
    'appetizer,main,salad,side': 'main', 
    'appetizer,salad': 'salad', 
    'appetizer,side': 'appetizer',
    'appetizer,soups_and_stews': 'soups_and_stews', 
    'bread': 'bread', 
    'bread,main': 'bread', 
    'dessert': 'dessert',
    'dessert,main': 'dessert', 
    'dessert,salad': 'dessert', 
    'drinks': 'drinks', 
    'main': 'main', 
    'main,salad': 'salad',
    'main,side': 'side', 
    'main,side,soups_and_stews': 'main', 
    'main,soups_and_stews': 'main',
    'salad': 'salad', 
    'salad,side': 'salad', 
    'side': 'side', 
    'side,soups_and_stews': 'soups_and_stews',
    'soups_and_stews': 'soups_and_stews',
    ',main': 'main'
}
data = data.replace({'meal_type': fix_meal_type_dict})
fix_region_dict = {
    'african': 'african', 
    'african,european': 'african',
    'african,european,usa': 'african',
    'african,middle_eastern': 'african', 
    'african,middle_eastern,usa': 'african',
    'african,usa': 'african', 
    'asian': 'asian', 
    'asian,austrailia_new_zealand,usa': 'asian',
    'asian,european': 'asian', 
    'asian,usa': 'asian', 
    'austrailia_new_zealand': 'austrailia_new_zealand',
    'austrailia_new_zealand,european': 'austrailia_new_zealand', 
    'austrailia_new_zealand,usa': 'austrailia_new_zealand',
    'european': 'european',
    'european,latin_american': 'european', 
    'european,middle_eastern': 'european',
    'european,middle_eastern,usa': 'european', 
    'european,usa': 'european', 
    'latin_american': 'latin_american',
    'latin_american,usa': 'latin_american', 
    'middle_eastern': 'middle_eastern', 
    'middle_eastern,usa': 'middle_eastern',
    'usa': 'usa'
}
data = data.replace({'region': fix_region_dict})

# Load images, split data into train and test

In [7]:
images = []
imgzip = zipfile.ZipFile("Recipe_Images_224_x_224.zip")
for f in tqdm(imgzip.infolist()):
    ifile = imgzip.open(f)
    img = np.asarray(Image.open(ifile))
    images.append(img)


100%|██████████| 8573/8573 [00:13<00:00, 614.67it/s]


In [8]:
train_df, test_df = train_test_split(data, test_size = 0.2, random_state = SEED)

In [9]:
train_df.head()

,id,servings,calories_per_serving,protein_per_serving,fat_per_serving,carb_per_serving,meal_type,region
4971,8160,4,127,4,9,9,salad,european
2573,4249,6,300,6,12,44,side,asian
786,1354,8,395,11,17,48,UNK_meal_type,european
3684,6038,25,318,26,6,41,UNK_meal_type,european
435,784,35,141,7,8,11,UNK_meal_type,latin_american


In [10]:
train_idx = train_df['id'].values
test_idx = test_df['id'].values
train_images = np.array([images[idx] for idx in train_idx])
test_images = np.array([images[idx] for idx in test_idx])

In [11]:
train_y = np.column_stack(
    (train_df['calories_per_serving'].values,
     train_df['protein_per_serving'].values,
     train_df['fat_per_serving'].values,
     train_df['carb_per_serving'].values)
)
test_y = np.column_stack(
    (test_df['calories_per_serving'].values,
     test_df['protein_per_serving'].values,
     test_df['fat_per_serving'].values,
     test_df['carb_per_serving'].values)
)

In [12]:
one_hot_enc = OneHotEncoder(sparse = False)
train_df_tabular = train_df[['meal_type', 'region']]
train_tabular = one_hot_enc.fit_transform(train_df_tabular)
test_df_tabular = test_df[['meal_type', 'region']]
test_tabular = one_hot_enc.transform(test_df_tabular)

# Create model

In [13]:
IMG_SHAPE = (224, 224, 3)

In [14]:
def create_cnn(IMG_SHAPE, fine_tune=False, fine_tune_at=0):
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomFlip('horizontal'),
        tf.keras.layers.RandomRotation(0.2),
    ])
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)
    base_model.trainable = False
    if fine_tune:
        fine_tune_at = fine_tune_at
        for layer in base_model.layers[-fine_tune_at:]:
            if not isinstance(layer, layers.BatchNormalization):
                layer.trainable = True

    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    x = base_model(x, training = False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    outputs = Dense(4, activation='relu')(x)
    model = Model(inputs, x)
    return model


def create_mlp(dim):
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation='relu'))
    model.add(Dense(4, activation='relu'))
    return model

In [15]:
mlp = create_mlp(train_tabular.shape[1])
cnn = create_cnn(IMG_SHAPE)

combined_input = concatenate([mlp.output, cnn.output])

x = Dense(8, activation = 'relu')(combined_input)
x = Dense(4, activation = 'linear')(x)

model = tf.keras.Model(inputs = [mlp.input, cnn.input], outputs = x)
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
    loss = 'huber_loss',
    metrics = ['mean_absolute_error']
)
model.summary()

29084464/29084464 [==============================] - 2s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential_1 (Sequential)      (None, 224, 224, 3)  0           ['input_2[0][0]']                
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 224, 224, 3)  0           ['sequential_1[0][0]']           
                                                                                                  
 tf.nn.bias_add (TFOpLambda

In [16]:
es = EarlyStopping(monitor='val_mean_absolute_error', patience=3, restore_best_weights=True)
initial_epochs = 100
history = model.fit(
    x = [train_tabular, train_images],
    y = train_y,
    batch_size = 32,
    epochs = initial_epochs,
    validation_split = 0.2,
    callbacks = [es]
)
model.save_weights('initial_multiin_multiout_densenet.h5')

Epoch 1/100
105/105 [==============================] - 33s 169ms/step - loss: 94.8093 - mean_absolute_error: 95.3008 - val_loss: 90.1132 - val_mean_absolute_error: 90.6071
Epoch 2/100
105/105 [==============================] - 14s 137ms/step - loss: 87.0982 - mean_absolute_error: 87.5922 - val_loss: 82.7260 - val_mean_absolute_error: 83.2199
Epoch 3/100
105/105 [==============================] - 14s 133ms/step - loss: 75.4045 - mean_absolute_error: 75.8972 - val_loss: 66.8549 - val_mean_absolute_error: 67.3483
Epoch 4/100
105/105 [==============================] - 14s 138ms/step - loss: 57.4985 - mean_absolute_error: 57.9916 - val_loss: 52.5856 - val_mean_absolute_error: 53.0792
Epoch 5/100
105/105 [==============================] - 14s 132ms/step - loss: 50.6172 - mean_absolute_error: 51.1098 - val_loss: 50.7373 - val_mean_absolute_error: 51.2300
Epoch 6/100
105/105 [==============================] - 14s 137ms/step - loss: 49.9015 - mean_absolute_error: 50.3946 - val_loss: 50.3724 - v

In [ ]:
test_preds = model.predict([test_tabular, test_images])
mae_cal = mean_absolute_error(test_y[:,0:1], test_preds[:,0:1])
mae_pro = mean_absolute_error(test_y[:,1:2], test_preds[:,1:2])
mae_fat = mean_absolute_error(test_y[:,2:3], test_preds[:,2:3])
mae_carb = mean_absolute_error(test_y[:,3:4], test_preds[:,3:4])
print(mae_cal, mae_pro, mae_fat, mae_carb)

33/33 [==============================] - ETA: 0s

In [ ]:
runtime.unassign()